<a href="https://colab.research.google.com/github/dgeip/kayip_satis/blob/main/kayip_satis_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lost Sale project


The Lost Sale project I managed and coded while interning as a Data Analyst at MUDO, a retail company. The lost sales project is a machine learning project that estimates the 'demand gap' caused by stock shortages in the past. In the end, lost sales are displayed based on the store, size, and color. This can be utilized for purchasing of the product and allocation in the following year."

In this notebook, linen shirt for summer 2022 named GM61 is examined.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
piece = input(str("Name of the piece: "))

Name of the piece: GM61


In [ ]:
# Import dataset
data = pd.read_excel(str(piece+"DB.xlsx"))

In [ ]:
# Select the season period of the dataset
input(str("Sezon(Y/K): "))
if input == "Y" or input == "y":
  data = data[(17 <= data.Hafta) & ( data.Hafta < 35)]
elif input == "K" or input == "k":
  data = data[(data.Hafta < 17) & (data.Hafta > 35)]

Sezon(Y/K): y


In [ ]:
# Create the "Key" column
data["Key"] = data["Birim"] + data["Beden"] + data["Renk"]

In [ ]:
# Detection and drop of the "Birim" where the product never went
grouped = data.groupby("Key")

for name, group in grouped:
  if np.sum(group.Stok) == 0:
    data = data.drop(group.index)

In [ ]:
# Function for Moving Average columns
def create_ma_columns(data, col_name, window_sizes):

    for window_size in window_sizes:
        col = f"{col_name}_MA_{window_size}"
        data[col] = data.groupby('Key')[col_name].transform(lambda x: x.rolling(window=window_size).mean())
    return data


In [ ]:
# Creation of 3-week MA columns
df = create_ma_columns(data, "Satış", [3])

In [ ]:
df

,Hafta,Birim,Beden,Renk,Key,Satış,Stok,Satış_MA_3
0,17,A.pazarı Agora Mudo Giyim,L,BEYAZ,A.pazarı Agora Mudo GiyimLBEYAZ,1,1,NaN
1,17,A.pazarı Agora Mudo Giyim,L,SİYAH,A.pazarı Agora Mudo GiyimLSİYAH,0,2,NaN
2,17,A.pazarı Agora Mudo Giyim,L,GÜL KURUSU,A.pazarı Agora Mudo GiyimLGÜL KURUSU,0,2,NaN
3,17,A.pazarı Agora Mudo Giyim,L,KİREMİT,A.pazarı Agora Mudo GiyimLKİREMİT,0,2,NaN
4,17,A.pazarı Agora Mudo Giyim,L,AÇIK MAVİ,A.pazarı Agora Mudo GiyimLAÇIK MAVİ,0,2,NaN
...,...,...,...,...,...,...,...,...
187901,52,Yalova Setur Marina,XXL,SİYAH,Yalova Setur MarinaXXLSİYAH,0,0,0.0
187906,52,Yalova Setur Marina,XXL,İNDİGO,Yalova Setur MarinaXXLİNDİGO,0,0,0.0
187910,52,Yalova Setur Marina,XXXL,BEYAZ,Yalova Setur MarinaXXXLBEYAZ,0,0,0.0
187911,52,Yalova Setur Marina,XXXL,SİYAH,Yalova Setur MarinaXXXLSİYAH,0,0,0.0


In [ ]:
# Create a new column to fill in "0" sales due to lost sales and NaN values ​due to the MA_3
df['gen_MA_1'] = df['Satış_MA_3'].copy()

In [ ]:
def plot_dist(best_distribution, best_param):
    print(f"Best fitting distribution: {best_distribution.name}")
    print(f"Parameters: {best_params}")
    plt.hist(data, bins=20, density=True, alpha=0.6, color='g')
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = best_distribution(*best_param).pdf(x)
    plt.plot(x, p, 'k', linewidth=2)
    title = f"{best_distribution.name}"
    plt.title(title)
    plt.show()

In [ ]:
from scipy import stats


np.random.seed(42)

# Distributions
distributions = [stats.norm, stats.expon, stats.lognorm, stats.beta, stats.chi2,
                 stats.gamma, stats.genextreme, stats.genpareto, stats.triang]

best_distributions = {}
best_params = {}
best_sse = {}

# Create grouped data on the basis of "Renk", since distributions will be tried on "Renk"
grouped = df.groupby("Renk")

# Iterate over each group
for name, group in grouped:

    # Drop NaN values ​​to make distribution fit work
    data = group['gen_MA_1'].dropna()

    best_distribution = None
    best_param = None
    best_sse = np.inf

    # Try distributions
    for distribution in distributions:
        try:
            params = distribution.fit(data)

            # Calculate SSE
            sse = np.sum((data - distribution(*params).rvs(len(data), random_state=42)) ** 2)

            # Check if it is better than the previous iteration
            if sse < best_sse:
                best_distribution = distribution
                best_param = params
                best_sse = sse
        except Exception as e:
            print(f"Could not fit {distribution.name} distribution for group {name}: {e}")

    # Store the optimal distribution and parameters for the current group
    best_distributions[name] = best_distribution
    best_params[name] = best_param

    # Get the optimal distribution for the current group
    best_distribution = best_distributions[name]
    best_param = best_params[name]

    # (Optional) Plotting the distribution of "Renk"
    # plot_dist(best_distribution, best_param)

    # Fill NaN values ​by randomly selecting values from distribution
    missing_indexes = group.index[group['gen_MA_1'].isna()]
    for missing_index in missing_indexes:
        df.loc[missing_index, 'gen_MA_1'] = best_distribution(*best_param).rvs()

    # Fill "0" values ​by randomly selecting values from distribution
    zero_indexes = group[(group.Satış_MA_3 == 0) & (group.Stok == 0)].index
    for zero_index in zero_indexes:
        df.loc[zero_index, 'gen_MA_1'] = best_distribution(*best_param).rvs()


In [ ]:
print(f"Sum of the original Satış_MA3: {df.Satış_MA_3.sum()}")
print(f"Sum of the Generated Satış_MA3: {df.gen_MA_1.sum()}")

Sum of the original Satış_MA3: 10519.666666666666
Sum of the Generated Satış_MA3: 13238.755607286674


In [ ]:
# Drop the "Key" and "Original Satış_MA_3" columns
df = df.drop(["Satış_MA_3", "Key"], axis = 1)

In [ ]:
## (OPTIONAL) Correlation heatmap
# sns.heatmap(df.drop("Birim", axis = 1).corr(), annot = True);

In [ ]:
# Check for missing values
df.isna().sum()

Hafta       0
Birim       0
Beden       0
Renk        0
Satış       0
Stok        0
gen_MA_1    0
dtype: int64

In [ ]:
# Suppression of outliers
df.Satış[df.Satış >= 6 ] = 5
df.Satış[df.Satış <= 0 ] = 0

In [ ]:
# Get the preliminary schema for Excel output
out_excel= df[df.Stok == 0]

In [ ]:
# Create the dummy variables
df = pd.get_dummies(df, drop_first = True)

In [ ]:
# Create the test set
test_df = df[df.Stok == 0]

# Select the places where inventory is not 0 or where Sales is greater than zero for the train dataset
# Thus, places where "stock and sales are 0 at the same time" are not available in the dataset.

df = df[(df.Stok != 0)|(df.Satış > 0)]

In [ ]:
X_train = df.drop(["Satış", "Stok"], axis = 1)
y_train = df.Satış

In [ ]:
X_test = test_df.drop(["Satış", "Stok"], axis = 1)
y_test = test_df.Satış

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, f1_score

In [ ]:
# Standardization with Standard Scaler (-1,1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Implementation of SelectKBest for Feature Selection
selector = SelectKBest(k = 45)
selector.fit(X_train_scaled, y_train)
X_train_selected = selector.transform(X_train_scaled)

In [ ]:
# XGBRegressor
model = XGBRegressor(random_state = 42)

# Establish the ML pipeline to prevent data leakage
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(k=45)),
    ('model', model)
])

In [ ]:
# Fit the model
pipe.fit(X_train, y_train)

# Predictions
y_pred = pipe.predict(X_train)
y_pred = np.around(y_pred)

# Evaluation
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred, average = "macro")
print(f'Mean Squared Error: {mse}')
print(f'F1 Score: {f1}')
print(f'R2 Score: {r2}')

Mean Squared Error: 0.15213829865271292
F1 Score: 0.5573113758421842
R2 Score: 0.494596342157824


In [ ]:
print(f"Sum of the predictions based on train set: {y_pred.sum()}")
print(f"Sum of the Satış in the train set : {y_train.sum()}")

Sum of the predictions based on train set: 9509.0
Sum of the Satış in the train set : 10883


In [ ]:
# Cross Validation
# StratifiedKFold is used instead of plain KFold because dataset is "unbalanced"
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Create the CV scores and examine the average
ffcv = cross_val_score(pipe, X_train, y_train, cv= skf, scoring = "r2")
print(f"Ortalama CV Score: {np.mean(ffcv)}")
print(f"CV Skorları: {ffcv}")

Ortalama CV Score: 0.5068540599460543
CV Skorları: [0.49831345 0.51296057 0.48832121 0.51549645 0.51917862]


In [ ]:
# Create a dictionary to select the optimum "k" value for SelectKBest
param_grid = {
        'selector__k': [10,20,30,40,45,50]
}

# Select the optimum "k" value with GridSearch
grid = GridSearchCV(pipe, param_grid, cv=skf, scoring='r2', n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector', SelectKBest(k=45)),
                                       ('model',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     early_stopping_rounds=None,
                                                     enable_categorical=False,
                                                     eval_me...
                                                     learning_rate=None,
                                                     max_bin=None,
                                                     max_cat_threshold=None,
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=42, ...))]),
             n_jobs=-1, param_grid={'selector__k': [10, 20, 30, 40, 45, 50]},
             scoring='r2')

In [ ]:
print(f'Best Parameters: {grid.best_params_}')
print(f'Best R2: {grid.best_score_}')

Best Parameters: {'selector__k': 30}
Best R2: 0.5071517137818191


In [ ]:
# Redefine the pipeline with the best k value
pipe = grid.best_estimator_

In [ ]:
# Train the model
pipe.fit(X_train, y_train)

# Make predictions
y_pred = pipe.predict(X_train)
y_pred = np.around(y_pred)

# Evaluate the performance of the model
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred, average = "macro")
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')
print(f'F1 Score: {f1}')

Mean Squared Error: 0.15793334474411325
R2 Score: 0.4753451902920902
F1 Score: 0.5755858410731509


In [ ]:
print(f"Sum of the new predictions based on train set: {y_pred.sum()}")
print(f"Sum of the train set: {y_train.sum()}")

Sum of the new predictions based on train set: 9532.0
Sum of the train set: 10883


In [ ]:
ffcv = cross_val_score(pipe, X_train, y_train, cv= skf, scoring = "r2")
print(f"New Average CV Score: {np.mean(ffcv)}")
print(f"New CV Scores: {ffcv}")

New Average CV Score: 0.5071517137818191
New CV Scores: [0.49791266 0.52271482 0.49781697 0.51018103 0.5071331 ]


In [ ]:
## TEST SET PREDICTION

In [ ]:
# Predictions
y_pred_test = pipe.predict(X_test)
y_pred_test = np.around(y_pred_test)

In [ ]:
print(f" Sum of the test set: {y_test.sum()}")
print(f" Sum of the predictions based on test set: {y_pred_test.sum()}")

 Sum of the test set: 5531
 Sum of the predictions based on test set: 10295.0


In [ ]:
out_excel

,Hafta,Birim,Beden,Renk,Satış,Stok,gen_MA_1
19,17,A.pazarı Agora Mudo Giyim,M,KEKİK,2,0,9.301005e-03
27,17,A.pazarı Agora Mudo Giyim,S,YEŞİL,1,0,1.799006e-08
33,17,A.pazarı Agora Mudo Giyim,XL,KİREMİT,1,0,6.731367e-02
36,17,A.pazarı Agora Mudo Giyim,XL,İNDİGO,1,0,1.196076e-09
39,17,A.pazarı Agora Mudo Giyim,XL,KEKİK,1,0,2.198622e-02
...,...,...,...,...,...,...,...
187901,52,Yalova Setur Marina,XXL,SİYAH,0,0,7.860120e-02
187906,52,Yalova Setur Marina,XXL,İNDİGO,0,0,1.277042e-10
187910,52,Yalova Setur Marina,XXXL,BEYAZ,0,0,3.162078e-02
187911,52,Yalova Setur Marina,XXXL,SİYAH,0,0,2.924529e-03


In [ ]:
# Add the prediction column named "pred" to the prepared template
out_excel["pred"] = y_pred_test

In [ ]:
# Add the Kayıp Satış ("KS") column max(0, (Prediction - Obsorved))
out_excel['KS'] = out_excel.apply(lambda row: max(0, row['pred']-row['Satış']), axis=1)

In [ ]:
print(f" Sum of the Lost sales: {out_excel.KS.sum()}")

 Sum of the Lost sales: 6251.0


In [ ]:
# Export the Excel file
out_excel.to_excel(str("KS_"+piece+".xlsx"))